### 1. Imports & Environment Setup

In [ ]:
import json
import time
import pandas as pd
import os
import yaml
import requests
import weaviate
import re
from tqdm import tqdm
from dotenv import load_dotenv
from weaviate.classes.init import Auth, AdditionalConfig, Timeout
from sentence_transformers import CrossEncoder
from rouge_score import rouge_scorer
from typing import List, Dict, Tuple

load_dotenv()
print("✅ Environment initialized.")

### 2. Configuration and Prompt Loading

In [ ]:
CONFIG_DIR = "../src/config/"

# Load Configuration
with open(os.path.join(CONFIG_DIR, "config.yaml"), "r") as f:
    config = yaml.safe_load(f)

# Load Prompts
with open(os.path.join(CONFIG_DIR, "prompts.yaml"), "r") as f:
    prompts = yaml.safe_load(f)

print(f"✅ Config & Prompts Loaded.")

### 3. Judge Functions (ROUGE & LLM Evaluation)

In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def calculate_rouge(pred, ref):
    """Calculates ROUGE-L F-measure between prediction and ground truth."""
    if not pred or pred in ["N/A", "Error"]: return 0.0
    return scorer.score(ref, pred)['rougeL'].fmeasure

def llm_judge(question, answer, truth):
    """Uses an LLM to rate accuracy on a scale of 1-5."""
    if not answer or answer == "N/A": return 1
    
    provider = config['evaluation']['judge_provider']
    settings = config['providers'][provider]
    model_name = settings['judge_model']
    
    prompt = prompts['judge_prompt'].format(
        question=question, truth=truth, answer=answer
    )
    
    try:
        if "ollama" in provider:
            response = requests.post(settings['base_url'], json={
                "model": model_name, "prompt": prompt, "stream": False, 
                "options": {"temperature": 0.0}}, timeout=30)
            text = response.json()['response'].strip()
        else:
            api_key = os.getenv(f"{provider.upper()}_API_KEY") or os.getenv("HF_TOKEN")
            response = requests.post(settings['base_url'], 
                headers={"Authorization": f"Bearer {api_key}"},
                json={"model": model_name, "messages": [{"role": "user", "content": prompt}], "temperature": 0.0},
                timeout=10)
            text = response.json()['choices'][0]['message']['content'].strip()

        match = re.search(r'\[\[([1-5])\]\]', text) or re.search(r'\b([1-5])\b', text)
        return int(match.group(1)) if match else 1
    except:
        return 1

print("✅ Evaluation functions loaded.")

### 4. Weaviate, Reranker Setup & Advanced RAG with RRF

In [ ]:
print("🔌 Connecting to Weaviate...")
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=Auth.api_key(os.getenv("WEAVIATE_API_KEY")),
    headers={"X-HuggingFace-Api-Key": os.getenv("HF_TOKEN")},
    additional_config=AdditionalConfig(timeout=Timeout(init=60, query=60))
)

print("📚 Loading Cross-Encoder Reranker...")
reranker = CrossEncoder(config['weaviate']['reranker_model'])

def reciprocal_rank_fusion(ranked_lists: List[List[Tuple[str, any]]], k: int = 60) -> Dict[str, float]:
    """
    Implements Reciprocal Rank Fusion (RRF) algorithm.
    
    RRF Score Formula: RRF(d) = Σ 1/(k + rank_i(d))
    
    Args:
        ranked_lists: List of ranked result lists from different retrievers
        k: RRF constant parameter (default: 60)
    
    Returns:
        Dictionary mapping doc_id to RRF score and document object
    """
    rrf_scores = {}
    
    for ranked_list in ranked_lists:
        for rank, (doc_id, doc_obj) in enumerate(ranked_list):
            if doc_id not in rrf_scores:
                rrf_scores[doc_id] = {'score': 0.0, 'doc': doc_obj}
            rrf_scores[doc_id]['score'] += 1.0 / (k + rank + 1)
    
    return rrf_scores

def query_librarian(q: str, top_k: int = 10, rrf_k: int = 60) -> str:
    """
    Advanced RAG Pipeline with Reciprocal Rank Fusion.
    
    Pipeline:
    1. Dense Vector Search (Semantic)
    2. BM25 Keyword Search (Lexical)
    3. Reciprocal Rank Fusion
    4. Cross-Encoder Reranking
    
    Args:
        q: Query string
        top_k: Number of candidates per retriever
        rrf_k: RRF constant
    
    Returns:
        Most relevant document content
    """
    try:
        col = client.collections.get(config['weaviate']['class_name'])
        
        # Step 1: Dense Vector Search
        vector_results = col.query.near_text(query=q, limit=top_k * 2)
        
        # Step 2: BM25 Keyword Search
        bm25_results = col.query.bm25(query=q, limit=top_k * 2)
        
        # Check if we have results
        if not vector_results.objects and not bm25_results.objects:
            return "N/A"
        
        # Step 3: Apply Reciprocal Rank Fusion
        vector_ranked = [(str(obj.uuid), obj) for obj in vector_results.objects]
        bm25_ranked = [(str(obj.uuid), obj) for obj in bm25_results.objects]
        
        rrf_scores = reciprocal_rank_fusion(
            ranked_lists=[vector_ranked, bm25_ranked],
            k=rrf_k
        )
        
        # Sort by RRF score
        rrf_sorted = sorted(
            rrf_scores.items(),
            key=lambda x: x[1]['score'],
            reverse=True
        )[:top_k]
        
        # Extract candidates
        rrf_candidates = [doc_data['doc'] for doc_id, doc_data in rrf_sorted]
        candidate_texts = [obj.properties['content'] for obj in rrf_candidates]
        
        # Step 4: Cross-Encoder Reranking
        pairs = [[q, text] for text in candidate_texts]
        scores = reranker.predict(pairs)
        
        # Return best match
        return candidate_texts[scores.argmax()]
        
    except Exception as e:
        print(f"⚠️ Error in query_librarian: {str(e)}")
        return "N/A"

print("✅ Librarian Online with RRF Pipeline.")

### 5. Evaluation Arena - The Showdown


In [ ]:
# Path configuration
final_output_path = config['paths']['final_output_path']
checkpoint_path = config['paths']['checkpoint_path']

# Load The Intern's predictions
print("📂 Loading The Intern's predictions...")
with open(config['paths']['intern_preds'], "r") as f:
    intern_lookup = {item['question']: item for item in [json.loads(line) for line in f]}

# Load golden test set
print("📂 Loading golden test set...")
with open(config['paths']['test_data'], "r") as f:
    test_set = [json.loads(line) for line in f] 

# Initialize results tracking
results = []
processed_questions = set()

# Resume from checkpoint if exists
if os.path.exists(checkpoint_path):
    existing_df = pd.read_csv(checkpoint_path)
    results = existing_df.to_dict('records')
    processed_questions = set(existing_df["Question"].tolist())
    print(f"🔄 Resuming from checkpoint. Skipping {len(processed_questions)} items.")

print(f"\n⚔️ Starting The Showdown ({len(test_set)} items total)")
print("=" * 80)
print("Arena Rules:")
print("  • The Librarian: Advanced RAG with RRF + Cross-Encoder Reranking")
print("  • The Intern: Fine-Tuned Llama-3-8B with LoRA")
print("  • Metrics: ROUGE-L, LLM-as-Judge (1-5), Latency (ms)")
print("=" * 80)

# Main evaluation loop
for item in tqdm(test_set, desc="Evaluating"):
    q = item['instruction']
    
    # Skip if already processed
    if q in processed_questions:
        continue
    
    truth = item['output']
    
    #The Librarian's Turn
    start = time.time()
    try:
        lib_ans = query_librarian(q)
    except Exception as e:
        lib_ans = "Error"
        print(f"\n⚠️ Librarian error on question: {q[:50]}...")
    lib_lat = (time.time() - start) * 1000
    
    #The Intern's Turn
    rec = intern_lookup.get(q, {})
    int_ans = rec.get('intern_answer', "N/A")
    int_lat = rec.get('intern_latency_ms', 0.0)
    
    #Calculate Metrics
    l_rouge = calculate_rouge(lib_ans, truth)
    i_rouge = calculate_rouge(int_ans, truth)
    l_score = llm_judge(q, lib_ans, truth)
    i_score = llm_judge(q, int_ans, truth)
    
    #Record Results
    res_row = {
        "Question": q,
        "Librarian_Judge": l_score,
        "Intern_Judge": i_score,
        "Librarian_Rouge": l_rouge,
        "Intern_Rouge": i_rouge,
        "Librarian_Latency": lib_lat,
        "Intern_Latency": int_lat
    }
    results.append(res_row)
    
    # Save checkpoint after each item
    pd.DataFrame([res_row]).to_csv(
        checkpoint_path,
        mode='a',
        header=not os.path.isfile(checkpoint_path),
        index=False
    )
    
    # Rate limiting to avoid API throttling
    time.sleep(2)

# Save final results
df_final = pd.DataFrame(results)
df_final.to_csv(final_output_path, index=False)

# Clean up checkpoint
if os.path.exists(checkpoint_path):
    os.remove(checkpoint_path)

print(f"\n✅ Evaluation Complete! Results saved to {final_output_path}")
print(f"📊 Total items evaluated: {len(results)}")

### 6. Results Summary & Comparison Table

In [ ]:
import pandas as pd
import os

# Load results
if os.path.exists(final_output_path):
    report_df = pd.read_csv(final_output_path)
    status = f"✅ Final Showdown Report ({len(report_df)} Items)"
elif os.path.exists(checkpoint_path):
    report_df = pd.read_csv(checkpoint_path)
    status = "⚠️ Partial Showdown Report (In Progress)"
else:
    report_df = None
    print("❌ No results found. Please run the Evaluation Arena loop in Cell 5 first.")

if report_df is not None:
    print(f"--- {status} ---")
    print(f"Evaluation Pipeline: Vector Search → BM25 → RRF → Cross-Encoder Reranking")
    print("=" * 80)
    
    # Calculate wins
    lib_wins = (report_df['Librarian_Judge'] > report_df['Intern_Judge']).sum()
    int_wins = (report_df['Intern_Judge'] > report_df['Librarian_Judge']).sum()
    ties = (report_df['Librarian_Judge'] == report_df['Intern_Judge']).sum()

    # Create summary table
    summary_data = {
        "Metric": [
            "Avg Judge Score (1-5)",
            "Avg ROUGE-L (0-1)",
            "Avg Latency (ms)",
            "Total Wins (Judge)"
        ],
        "Librarian (RAG+RRF)": [
            round(report_df['Librarian_Judge'].mean(), 2),
            round(report_df.get('Librarian_Rouge', 0).mean(), 4),
            f"{round(report_df['Librarian_Latency'].mean(), 2)} ms",
            lib_wins
        ],
        "Intern (Fine-Tuned)": [
            round(report_df['Intern_Judge'].mean(), 2),
            round(report_df.get('Intern_Rouge', 0).mean(), 4),
            f"{round(report_df['Intern_Latency'].mean(), 2)} ms",
            int_wins
        ]
    }
    
    metrics_df = pd.DataFrame(summary_data)
    
    # Display results
    print("\n🏆 Consolidated Performance Summary:")
    print("=" * 80)
    display(metrics_df.rename(index=lambda x: x + 1))
    
    print(f"\nTotal Items Evaluated: {len(report_df)} | Total Ties: {ties}")
    
    # Determine winner
    if int_wins > lib_wins:
        winner = "🏆 The Intern (Fine-Tuned Model)"
        margin = int_wins - lib_wins
    elif lib_wins > int_wins:
        winner = "🏆 The Librarian (RAG with RRF)"
        margin = lib_wins - int_wins
    else:
        winner = "🤝 TIE"
        margin = 0
    
    print(f"\n{winner}")
    if margin > 0:
        print(f"Win Margin: {margin} questions ({margin/len(report_df)*100:.1f}%)")
    
    print("\n📊 Detailed Data Sample (Top 5):")
    display(report_df.head(5).rename(index=lambda x: x + 1))

### 7. BONUS: Monthly Cloud Cost Analysis

To provide a realistic financial comparison, this analysis estimates the monthly operational costs for serving both **"The Intern" (Fine-Tuning)** and **"The Librarian" (Advanced RAG with RRF)** architectures.

## Scale Assumptions

- **Daily Active Users (DAU)**: 500
- **Queries per User**: 10
- **Total Daily Queries**: 5,000
- **Total Monthly Volume**: 150,000 queries (30-day month)

---

## Strategy A: "The Intern" (Parametric Memory)

This strategy relies on serving the fine-tuned Llama-3-8b model. To maintain consistent performance for 500 users, a dedicated GPU instance is required to host the model 24/7.

| Component       | Service / Hardware              | Unit Cost (Est. 2026) | Monthly Cost |
|-----------------|--------------------------------|-----------------------|--------------|
| Model Hosting   | AWS EC2 g4dn.xlarge (NVIDIA T4) | $0.5260 / hr         | $383.98      |
| Data Storage    | Amazon EBS (125 GB NVMe SSD)    | Included in instance | $0.00        |
| Update Cost     | Compute for retraining (Min. 100 steps) | ~$25.00 / run | ~$25.00      |
| **Total Monthly** |                               |                       | **~$408.98** |

**Note**: While hosting is fixed, "The Intern" suffers from **parametric decay**. Each new financial report requires a full retraining cycle, adding compute costs and high engineering effort.

---

## Strategy B: "The Librarian" (Non-Parametric Memory with RRF)

The Advanced RAG strategy uses a modular infrastructure with RRF-enhanced retrieval. While it has more components, it allows for **real-time data updates** without the need for retraining.

| Component       | Service / Component             | Unit Cost (Est. 2026)     | Monthly Cost |
|-----------------|--------------------------------|---------------------------|--------------|
| Vector Database | Weaviate Cloud (Standard)      | $25.00 base + usage      | ~$35.00      |
| Embeddings      | Snowflake Arctic (150k queries) | $0.025 / 1M tokens       | ~$5.00       |
| Reranker        | Cross-Encoder (Hosted t3.medium) | $0.0416 / hr           | ~$30.00      |
| Base LLM Hosting | AWS EC2 g4dn.xlarge (Standard Llama-3) | $0.5260 / hr    | $383.98      |
| **Total Monthly** |                               |                           | **~$453.98** |

---

## Cost Comparison Summary

| Metric                | "The Intern" (Fine-Tuning) | "The Librarian" (RAG+RRF) |
|-----------------------|----------------------------|---------------------------|
| **Total Monthly Cost** | $408.98                   | $453.98                   |
| **Cost Per Query**     | $0.0027                   | $0.0030                   |
| **Data Update Cost**   | High (Requires full retraining) | Low (Real-time indexing) |
| **Factual Grounding**  | Low (Risk of Hallucination) | High (Direct Citations + RRF) |

---

## Economic Verdict

- **Short-Term Savings**: "The Intern" appears roughly **10-11% cheaper** in fixed monthly infrastructure costs.
  
- **Long-Term Value**: "The Librarian" with RRF is the **superior choice for Alpha-Yield Capital**. It avoids the recurring **"engineering tax"** and compute costs of retraining every time a new financial report is released, providing exact page citations and real-time data freshness essential for fintech applications.

- **RRF Advantage**: The Reciprocal Rank Fusion component adds negligible compute cost (<$5/month) but significantly improves retrieval precision for financial queries requiring both semantic understanding and exact entity matching.

**Bottom Line**: While fine-tuning may seem appealing on paper, the hidden costs of parametric decay, retraining cycles, and hallucination risks make Advanced RAG with RRF the more economically sound and operationally reliable solution for production financial systems.